[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fw-ai/cookbook/blob/main/examples/function_calling/fireworks_functions_qa.ipynb)

# Structured answers with Fireworks functions

Several real world applications of LLM require them to respond in a strucutred manner. This structured response could look like `JSON` or `YAML`. For e.g. answering research questions using arxiv along with citations. Instead of parsing the entire LLM response and trying to figure out the actual answer of the LLM vs the citations provided by the LLM, we can use function calling ability of the LLMs to answer questions in a structured way.

In this notebook, we demonstrate structured response generation ability of the Fireworks function calling model. We will build an application that can answer questions (along with citations) regarding the State of the Union speech of 2022.

# Setup

Install all the dependencies and import the required python modules.

In [ ]:
!pip install openai

In [ ]:
import os
import requests
import re
import openai

##  Download & Clean the Content

We are going to download the content using the python package `requests` and perform minor cleanup by removing several newlines. Even minimal cleanup should be good enough to obtain good results with the model.

In [ ]:
url = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
content = requests.get(url).content
content = str(content, "utf-8")

In [ ]:
# Some clean up
clean_content = content.replace("\n\n", "\n")

## Setup your API Key

In order to use the Fireworks AI function calling model, you must first obtain Fireworks API Keys. If you don't already have one, you can one by following the instructions [here](https://readme.fireworks.ai/docs/quickstart).

In [ ]:
client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = "YOUR_FW_API_KEY",
)
model_name = "accounts/fireworks/models/firefunction-v1"

## Define the Structure

Let's define the strucutre in which we want our model to responsd. The JSON structure for function calling follows the conventions of [JSON Schema](https://json-schema.org/). Here we define a structure with `answer` and `citations` field.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_answer_with_sources",
            "description": "Answer questions from the user while quoting sources.",
            "parameters": {
                "type": "object",
                "properties": {
                  "answer": {
                      "type": "string",
                      "description": "Answer to the question that was asked."
                  },
                  "sources": {
                      "type": "array",
                      "items": {
                          "type": "string",
                          "description": "Source used to answer the question"
                      }
                  }
                },
                "required": ["answer", "sources"],
            }
        }
    }
]
tool_choice = {"type": "function", "function": {"name":"get_answer_with_sources"}}

## Perform Sanity Test

Let's perform a sanity test by querying the speech for some basic information. This would ensure that our model setup is working correctly and the document is being processed correctly.

In [ ]:
mp
essages = [
    {"role": "system", "content": f"You are a helpful assistant who is given document with following content: {clean_content}."
     															"Please reply in succinct manner and be truthful in the reply."},
    {"role": "user", "content": "What did the president say about Ketanji Brown Jackson?"}
]


In [ ]:
chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.1
)

In [ ]:
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": " ",
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_fN2C7m2xaRrkZj0O9gV46eKb",
            "function": {
                "arguments": "{\"answer\": \"I recently nominated Ketanji Brown Jackson to serve on the United States Supreme Court. She is an outstanding candidate and I am confident that she will serve with distinction.\", \"sources\": [\"The White House\"]}",
                "name": "get_answer_with_sources"
            },
            "type": "function",
            "index": 0
        }
    ]
}


In [ ]:
agent_response = chat_completion.choices[0].message

messages.append(
    {
        "role": agent_response.role,
        "content": "",
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in agent_response.tool_calls
        ]
    }
)

## Using Function Calling in Conversation

Our model currently support multi-turn conversation when using function calling. You can reference previous completions generated by the model to ask more clarifying questions.

In [ ]:
messages.append(
    {
        "role": "user",
        "content": "What did he say about her predecessor?"
    }
)
next_chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.1
)

In [ ]:
print(next_chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_SJsaq3GUUK2ugOVlJ025flwH",
            "function": {
                "arguments": "{\"answer\": \"I am deeply saddened by the passing of Justice Breyer. He was a brilliant jurist and a true American hero. His legacy will live on in the work he did on the Supreme Court and in the lives he touched.\", \"sources\": [\"The White House\"]}",
                "name": "get_answer_with_sources"
            },
            "type": "function",
            "index": 0
        }
    ]
}


## Modifying the output format to more specific one

During the conversation, some questions might need a more flexible response format. We have flexibility to change that during the conversation.



In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_answer_with_countries",
            "description": "Answer questions from the user while quoting sources.",
            "parameters": {
                "type": "object",
                "properties": {
                  "answer": {
                      "type": "string",
                      "description": "Answer to the question that was asked."
                  },
                  "countries": {
                      "type": "array",
                      "items": {
                          "type": "string",
                      },
                      "description": "countries mentioned in the sources"
                  }
                },
                "required": ["answer", "countries"],
            }
        }
    }
]
tool_choice = {"type": "function", "function": {"name":"get_answer_with_countries"}}

In [ ]:
agent_response = next_chat_completion.choices[0].message

messages.append(
    {
        "role": agent_response.role,
        "content": "",
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in agent_response.tool_calls
        ]
    }
)

messages.append(
    {
        "role": "user",
        "content": "What did he say about human traffickers?"
    }
)

In [ ]:
chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.1
)

In [ ]:
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_0Qymk6ZFNVsdjXmOOsmMsYLW",
            "function": {
                "arguments": "{\"answer\": \"We are working with our partners in South and Central America to host more refugees and secure their own borders. This will help us crack down on human traffickers and reduce the flow of illegal immigrants into the United States.\", \"countries\": [\"South America\", \"Central America\"]}",
                "name": "get_answer_with_countries"
            },
            "type": "function",
            "index": 0
        }
    ]
}
